<a href="https://colab.research.google.com/github/defog-ai/sqlcoder/blob/main/defog-sqlcoder-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip3 install torch transformers bitsandbytes accelerate torch torchvision torchaudio bitsandbytes tensorflow-metal --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu



Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cpu
You should consider upgrading via the '/Users/elhayef/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [7]:
import torch
import math

print(torch.backends.mps.is_available()) #the MacOS is higher than 12.3+
print(torch.backends.mps.is_built()) #MPS is activated

True
True


In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [17]:
torch.cuda.is_available()
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")


if torch.cuda.is_available():
    print(torch.device("cuda")) 
elif torch.backends.mps.is_available():
    print(torch.device("mps"))
else:
    print(torch.device("cpu"))
    

tensor([1.], device='mps:0')
mps


In [ ]:
# use an A100 on Colab Pro (or any system with >30GB VRAM on your own machine) to load this in bf16
# if not available, use a GPU with minimum 20GB VRAM to load this in 8bit, or with minimum 12GB of VRAM to load in 4bit
# on Colab, works with a V100 but crashes on a T4

# downloading the model and then loading it to memory step takes around 10 minutes the first time. Be patient :)

In [21]:
model_name = "defog/sqlcoder"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    # torch_dtype=torch.bfloat16,
    # load_in_8bit=True,
    # load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    # use_mps=True,
    # device_map="auto",
    use_cache=True,
)

generation_config.json: 100%|██████████| 111/111 [00:00<00:00, 193kB/s]


In [22]:
question = "What products has the biggest fall in sales in 2022 compared to 2021? Give me the product name, the sales amount in both years, and the difference."

In [24]:
prompt = """### Instructions:
Your task is convert a question into a SQL query, given a Postgres database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
- When creating a ratio, always cast the numerator as float

### Input:
Generate a SQL query that answers the questioq `{question}`.
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Response:
Based on your instructions, here is the SQL query I have generated to answer the question `{question}`:
```sql
""".format(question=question)

In [25]:
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

In [ ]:
# excruciatingly slow on an V100 with 4bit quantization
# takes around 1-2 minutes per query
# on a single A100 40GB, takes ~10-20 seconds

In [27]:
inputs = tokenizer(prompt, return_tensors="pt")
generated_ids = model.generate(
    **inputs,
    num_return_sequences=1,
    eos_token_id=eos_token_id,
    pad_token_id=eos_token_id,
    max_new_tokens=400,
    do_sample=False,
    num_beams=5
)

In [ ]:
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

In [ ]:
torch.cuda.empty_cache()
# empty cache so that you do generate more results w/o memory crashing
# particularly important on Colab – memory management is much more straightforward
# when running on an inference service

In [ ]:
print(outputs[0].split("```sql")[-1].split("```")[0].split(";")[0].strip() + ";")